# Sistema de Recomendación

Una vez que hemos aplicado procesos de extracción, transformación y carga a nuestros datos, estamos listos para entrenar nuestro modelo de Machine Learning y desarrollar nuestro sistema de recomendación. Este modelo tendrá una relación ítem-ítem, lo que significa que toma un ítem que corresponde a un juego específico y, basándose en qué tan similar es este ítem al resto, recomienda juegos similares. En este caso, el input es un juego y el output es una lista de juegos recomendados. Para lograr esto, aplicaremos la similitud del coseno como métrica de similitud entre los juegos.

### Pre-procesamiento de Datos

Importamos Librerias

In [1]:
import pandas as pd
import scipy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

En nuestro modelo vamos a utilizar los conjuntos de datos **steam_games.csv** y **user_reviews.csv** y vamos a unirlo mediante el 'id_item' que corresponde al identificador único del juego. 

In [6]:
df_games = pd.read_csv('Data/Data-Limpia/steam_games.csv')
df_reviews = pd.read_csv('Data/Data-Limpia/user_reviews.csv')
df_modelo= pd.merge(df_games,df_reviews,left_on='id',right_on='item_id', how='inner')
df_modelo.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer,user_id,posted,item_id,helpful,recommend,sentiment_analysis
0,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Single-player,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1
1,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Multi-player,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1
2,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Steam Trading Cards,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1


Eliminarmos las columnas que no vamos a utilizar y solo dejamos los features que será de utilidad para el modelo

In [7]:
df_modelo.drop(['publisher','app_name','release_date','early_access','id','developer','user_id','posted','helpful','price','tags','specs'], axis=1, inplace=True)
df_modelo.head(3)

,genres,title,item_id,recommend,sentiment_analysis
0,Action,Carmageddon Max Pack,282010,True,1
1,Action,Carmageddon Max Pack,282010,True,1
2,Action,Carmageddon Max Pack,282010,True,1


Contamos los sentimientos positivos, negativos y neutrales para cada juegos y la cantidad de recomendaciones y no recomendaciones para cada juego

In [8]:
# Usar pd.crosstab para contar la cantidad de cada valor de sentimiento por título
conteo_sentimientos = pd.crosstab(df_modelo['title'], df_modelo['sentiment_analysis'])
conteo_recomendaciones = pd.crosstab(df_modelo['title'], df_modelo['recommend'])
# Renombrar las columnas para mayor claridad
conteo_sentimientos.columns = ['negative_reviews','neutral_reviews','positive_reviews']
conteo_recomendaciones.columns = ['no_recommend_count', 'recommend_count']
# Reiniciar el índice para obtener 'title' como columna
conteo_sentimientos.reset_index(inplace=True)
conteo_recomendaciones.reset_index(inplace=True)


Unimos los el contemos de sentimiento y recomendaciones a traves de la columna 'title' del datafram original.

In [9]:
df_modelo = pd.merge(df_modelo, conteo_sentimientos, on='title', how='inner')
df_modelo = pd.merge(df_modelo, conteo_recomendaciones, on='title', how='inner')
df_modelo.head(3)


,genres,title,item_id,recommend,sentiment_analysis,negative_reviews,neutral_reviews,positive_reviews,no_recommend_count,recommend_count
0,Action,Carmageddon Max Pack,282010,True,1,0,72,0,0,72
1,Action,Carmageddon Max Pack,282010,True,1,0,72,0,0,72
2,Action,Carmageddon Max Pack,282010,True,1,0,72,0,0,72
3,Action,Carmageddon Max Pack,282010,True,1,0,72,0,0,72
4,Action,Carmageddon Max Pack,282010,True,1,0,72,0,0,72


In [10]:
df_modelo.drop(['recommend','sentiment_analysis'], axis=1, inplace=True)
df_modelo.head(3)

,genres,title,item_id,negative_reviews,neutral_reviews,positive_reviews,no_recommend_count,recommend_count
0,Action,Carmageddon Max Pack,282010,0,72,0,0,72
1,Action,Carmageddon Max Pack,282010,0,72,0,0,72
2,Action,Carmageddon Max Pack,282010,0,72,0,0,72


Eliminamos duplicados y reindexamos los indices.

In [11]:
df_modelo.duplicated().sum()

16329175

In [12]:
df_modelo.drop_duplicates(inplace=True)
df_modelo.reset_index(drop=True, inplace=True)
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7710 entries, 0 to 7709
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   genres              7684 non-null   object
 1   title               7710 non-null   object
 2   item_id             7710 non-null   int64 
 3   negative_reviews    7710 non-null   int64 
 4   neutral_reviews     7710 non-null   int64 
 5   positive_reviews    7710 non-null   int64 
 6   no_recommend_count  7710 non-null   int64 
 7   recommend_count     7710 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 482.0+ KB


En nuestro conjunto de datos contamos con características de texto y numéricas que requieren ciertas transformaciones. Para las características de texto, utilizaremos **CountVectorizer** para convertirlas en una matriz de recuento de términos. Esta matriz representa la frecuencia de ocurrencia de cada término (palabra) en cada documento. **CountVectorizer** es una de las formas más simples de representar datos de texto en un formato numérico que los algoritmos de aprendizaje automático pueden entender y procesar. Las características numéricas se seleccionan y se normalizan utilizando **StandardScaler** de sklearn.

In [13]:
text_features = ['genres','title']
numeric_features = ['negative_reviews', 'neutral_reviews', 'positive_reviews','no_recommend_count','recommend_count']

Combinar características de texto en una sola cadena por juego


In [14]:
df_modelo['combined_text'] = df_modelo[text_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_modelo.head(3)

,genres,title,item_id,negative_reviews,neutral_reviews,positive_reviews,no_recommend_count,recommend_count,combined_text
0,Action,Carmageddon Max Pack,282010,0,72,0,0,72,Action Carmageddon Max Pack
1,Indie,Carmageddon Max Pack,282010,0,72,0,0,72,Indie Carmageddon Max Pack
2,Racing,Carmageddon Max Pack,282010,0,72,0,0,72,Racing Carmageddon Max Pack


Inicializar CountVectorizer para convertir texto en una matriz de recuentos de términos

In [15]:
count_vectorizer = CountVectorizer()
text_matrix = count_vectorizer.fit_transform(df_modelo['combined_text'])
text_matrix.shape

(7710, 3751)

Normalizar características numéricas

In [16]:
scaler = StandardScaler()
numeric_matrix = scaler.fit_transform(df_modelo[numeric_features])
numeric_matrix.shape

(7710, 5)

### Calcular Similitud del Coseno

Combinar matrices de características de texto y numéricas. Usamos el modulo scipy.sparse que optimiza los calculos para matrices que tienen en su mayoria ceros.

In [17]:
combined_matrix = scipy.sparse.hstack((text_matrix, numeric_matrix))
combined_matrix.shape

(7710, 3756)

Calcular la similitud del coseno entre juegos basándose en la matriz combinada


In [18]:
cosine_sim = cosine_similarity(combined_matrix, combined_matrix)

Función para obtener juegos recomendados para un juego específico


In [25]:
def recomendacion_juego(item_id):
    # Obtener el índice del juego correspondiente al nombre dado
    idx = df_modelo[df_modelo['item_id'] == item_id].index[0]
    # Calcular la similitud del juego con todos los demás juegos en la muestra
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar los juegos según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos más similares (excluyendo el juego dado)
    sim_scores = [score for score in sim_scores if df_modelo.iloc[score[0]]['item_id'] != item_id][:5]

    # Obtener los nombres de los juegos recomendados (excluyendo repeticiones)
    recommended_games = []
    seen_games = set()
    for score in sim_scores:
        game_name = df_modelo.iloc[score[0]]['title']
        if game_name not in seen_games:
            recommended_games.append(game_name)
            seen_games.add(game_name)

    return recommended_games



In [26]:
item_id = 282010
recomendacion_juego(item_id)

['Kombat Pack',
 'Max Payne',
 'Max Payne 3',
 'Mad Max',
 'Max Payne 3: Deathmatch Made In Heaven Pack']